In [14]:
import os
import glob
import pandas as pd

In [15]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/Pax7Counts'

In [16]:
# Add experiment date here to apply to dataframe
analysis_date = '20181202'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in Image name column
    (df['ExptDate'], df['Treatment'], df['Dose'], df['Stains'], df['Embryo'], 
        df['Somites'], df['ImageMag'], df['Section']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Slice,Count,Total Area,Average Size,%Area,Mean,IntDen,Image,ExptDate,Treatment,Dose,Stains,Embryo,Somites,ImageMag,Section
0,CntlSide,24,3012.334,125.514,38.506,255,32006.052,20180616_SMPD3gRNA1DNA_2ugul_Pax7LamininTCFLef...,20180616,SMPD3gRNA1DNA,2ugul,Pax7LamininTCFLef,Emb2,8ss,20x,sec3
1,ExptSide,14,2088.058,149.147,38.272,255,38032.493,20180616_SMPD3gRNA1DNA_2ugul_Pax7LamininTCFLef...,20180616,SMPD3gRNA1DNA,2ugul,Pax7LamininTCFLef,Emb2,8ss,20x,sec3
0,CntlSide,23,1710.402,74.365,33.297,255,18963.153,"20181119_SMPD3MO_1,0mM_Pax7Snai2pbetacat_Emb5_...",20181119,SMPD3MO,"1,0mM",Pax7Snai2pbetacat,Emb5,6ss,20x,sec2
1,ExptSide,15,652.563,43.504,18.190,255,11093.575,"20181119_SMPD3MO_1,0mM_Pax7Snai2pbetacat_Emb5_...",20181119,SMPD3MO,"1,0mM",Pax7Snai2pbetacat,Emb5,6ss,20x,sec2
0,CntlSide,12,1031.405,85.950,13.740,255,21917.348,"20181102_SMPD3gRNA1RNP_0,35ugul_Pax7Laminin_Em...",20181102,SMPD3gRNA1RNP,"0,35ugul",Pax7Laminin,Emb2,8ss,40x,sec3


In [18]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Mean counts across sections
mean_sections = pd.DataFrame((full_df.groupby(['Treatment', 'Embryo', 'Slice', 'ExptDate'])['Count']).mean())

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(mean_sections.xs(treatment))

    # Extract means for Cntl and Expt counts
    counts_cntl = df_treatment.xs('CntlSide', level='Slice')['Count'] 
    counts_expt = df_treatment.xs('ExptSide', level='Slice')['Count']

    # Generate ratios as Expt/Cntl
    counts_ratios = pd.DataFrame(counts_expt / counts_cntl)
    counts_ratios.columns = ['Expt/Cntl Counts']

    # Normalize individual values to mean of control group
    norm_cntl = counts_cntl/(float(counts_cntl.mean()))
    norm_expt = counts_expt/(float(counts_cntl.mean()))

    # Combine processed values into single dataframe and output as csv file
    counts_cntl = pd.DataFrame(counts_cntl)
    counts_cntl.columns = ['Cntl Counts']
    counts_expt = pd.DataFrame(counts_expt)
    counts_expt.columns = ['Expt Counts']
    counts_ratios = pd.DataFrame(counts_ratios)
    counts_ratios.columns = ['Expt/Cntl Counts']
    norm_cntl = pd.DataFrame(norm_cntl)
    norm_cntl.columns = ['Norm Cntl Counts']
    norm_expt = pd.DataFrame(norm_expt)
    norm_expt.columns = ['Norm Expt Counts']
    results = (pd.concat([counts_cntl, counts_expt, counts_ratios, norm_cntl, norm_expt], axis=1, sort=True)).reset_index()
    results['ID'] = results.ExptDate.str.cat(results.Embryo)
    
    # Save out results at CSV file, update file name
    results.to_csv(analysis_date + '_' + treatment + '_Pax7CountResults.csv')